## Лабораторная работа №2

#### Задание 1
Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

#### Ход работы:

Импортируем библиотеки и загружаем датасет

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import pandas as pd

file_path = "../Dataset.xls"
df = pd.read_excel(file_path)

df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


Разделение данных и нормализация

In [2]:
# Разделение на признаки (X) и целевую переменную (y)
X = df.drop('default payment next month', axis=1)
y = df['default payment next month']

# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Гиперпараметры:
   - C (регуляризация): Определяет силу регуляризации в модели SVM (Support Vector Machine / Машина опорных векторов); более низкое значение C увеличивает штраф за неправильную классификацию, что может привести к созданию простой модели с большей обобщающей способностью.
   - gamma (ядерный коэффициент): Определяет влияние одного тренировочного примера на другие; меньшие значения gamma означают более широкие гауссовы функции и приводят к более простым/прямым границам решения.
   - kernel (ядро): Определяет тип использованного ядра (линейное или RBF); линейное ядро работает линейно, тогда как RBF (радиально-базисная функция) способно моделировать сложные границы решений.

In [3]:
param_grid = {'C': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [0.1, 1, 10]}

Подбор гиперпараметров с помощью перекрестной проверки

In [4]:
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_score = grid.best_score_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.806 total time=   4.9s
[CV 2/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.817 total time=   5.0s
[CV 3/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.807 total time=   5.1s
[CV 4/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.809 total time=   5.2s
[CV 5/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.806 total time=   4.9s
[CV 1/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.787 total time=  10.8s
[CV 2/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.785 total time=  10.8s
[CV 3/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.785 total time=  10.7s
[CV 4/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.785 total time=  10.8s
[CV 5/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.785 total time=  10.8s
[CV 1/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.783 total time=   8.9s
[CV 2/5] END .C=0.01, gamma=0.1, kernel=sigmoid

[CV 2/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.688 total time=   6.2s
[CV 3/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.681 total time=   6.4s
[CV 4/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.700 total time=   6.1s
[CV 5/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.678 total time=   6.2s
[CV 1/5] END .......C=1, gamma=1, kernel=linear;, score=0.806 total time=  18.8s
[CV 2/5] END .......C=1, gamma=1, kernel=linear;, score=0.817 total time=  16.4s
[CV 3/5] END .......C=1, gamma=1, kernel=linear;, score=0.807 total time=  31.8s
[CV 4/5] END .......C=1, gamma=1, kernel=linear;, score=0.809 total time=  30.5s
[CV 5/5] END .......C=1, gamma=1, kernel=linear;, score=0.806 total time=  16.6s
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.784 total time=  35.7s
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.791 total time=  36.2s
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.785 total time=  35.6s
[CV 4/5] END ..........C=1, 

Вывод лучших гиперпараметров и оценка производительности модели на тестовом наборе

In [7]:
print("Лучшие гиперпараметры:", grid.best_params_)
best_svc = grid.best_estimator_
y_pred = best_svc.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

Лучшие гиперпараметры: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Точность модели: 0.8171666666666667
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4687
           1       0.67      0.33      0.44      1313

    accuracy                           0.82      6000
   macro avg       0.75      0.64      0.66      6000
weighted avg       0.80      0.82      0.79      6000

